In [ ]:
import os
import numpy as np
import env_ingredient_add as env
import hw
import trajectories as traj
from importlib import reload
from enum import Enum

def get_scene():
    if os.getcwd() == "/datasets/_deepnote_work/manipulation/project": 
        #scene = open("/work/manipulation/project/objects/environment_setup.yaml")
        scene = env.get_environment_set_up(no_scene=True,include_driver=True)
        xmls = [os.getcwd() + "/package.xml", "/work/manipulation/project/package.xml"]
    else:
        #scene = open("objects/environment_setup.yaml") # local setup
        scene = env.get_environment_set_up(no_scene=True,include_driver=True)
        xmls = [os.getcwd() + "/package.xml"]

    return scene

meshcat = hw.StartMeshcat()

In [ ]:
reload(env)
scenario = hw.LoadScenario(data=get_scene())


### trajectory 0: iiwa_painter

In [ ]:
reload(env)
reload(traj)
reload(hw)

# at present, trajectory blank doesn't work
diagram,sim = hw.init_diagram(meshcat,scenario)
# hw.print_diagram(diagram)

context = diagram.CreateDefaultContext()

controller = diagram.GetSubsystemByName("diff_ik_integrator")
plant = diagram.GetSubsystemByName("station").GetSubsystemByName("plant")


In [ ]:
# hw.print_diagram(diagram,output_file="planner.png")

In [ ]:
hw.fix_base_pos(controller.get_parameters(), 0*[False]+3*[True]) # z is locked to 25 cm.
# hw.create_painter_trajectory(diagram,meshcat)

hw.run_simulation(meshcat,sim,10)

### trajectory 1: simple move to bowl

In [ ]:
hw.run_simulation(meshcat,sim,20)

## scratch code

### gets the name of the robot's joints

In [ ]:

# plant = diagram.GetSubsystemByName("station").GetSubsystemByName("plant")

# for index in plant.GetJointIndices():
#     print(plant.get_joint(index).name())

In [ ]:


print(controller.get_parameters().get_joint_position_limits())

print(f"after running: {controller.get_parameters().get_joint_position_limits()}")

hw.run_simulation(meshcat,sim,10)